# Обязательная часть

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

# Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

search_url = 'https://habr.com/kek/v2/articles'
publication_url = 'https://habr.com/news/'

def get_data(query, pages=1):
  """
  Принимает: строку поискового запроса и количество страниц
  Возвращает: список словарей, содержащих <дата> - <заголовок> - <ссылка на материал> - <количество лайков>; но не <текст материала>
  """
  rows = []
  for page in range(1, pages+1):
    links = get_links_by_query(query, page)
    rows.extend(links)

  # Выкидываем дубляжи (по полю id)
  unique_ids = set()
  unique_data = []
  for item in rows:
    if item['id'] not in unique_ids:
        unique_data.append(item)
        unique_ids.add(item['id'])
  
  # Добавляем ссылки на страницы
  for item in unique_data:
    link = f"https://habr.com/news/{item['id']}"
    item['link'] = link
    # получаем заголовок и содержимое публикации
    publication_data = get_publication_content(link)
    item['content'] = publication_data['content']
    item['title'] = publication_data['title']

  return unique_data

def get_links_by_query(query, page):
  """
  Принимает: строку поискового запроса и номер страниц
  Возвращает: Ссылки на страницы, удовлетворяющие результату поиска.
  """
  params = {
    'query': query, # текстовый запрос
    'page': page, # номер страницы
    'perPage': 5 # количество результатов на страницу
  }
  # Как удобно работать со структурированным JSON, а не лазить по HTML-разметке :)
  res = requests.get(search_url, params).json()
  # Получаем структурированную информацию о каждой публикации; здесь есть почти всё, что нам нужно (не хватает только содержимого статьи)
  publication_refs = res.get('publicationRefs', [])

  # Список для хранения результатов
  publications = []
  # Проход по всем ключам словаря
  for value in publication_refs.values():
    # Извлечение необходимых данных
    id_ = value['id']
    time_published = value['timePublished']
    score = value['statistics'].get('score', None)  # Используем get для безопасного доступа

    # Добавление в результат
    publications.append({'id': id_, 'timePublished': time_published, 'score': score})
  
  return publications

def get_publication_content(link):
  """
  Принимает: ссылку на новость
  Возвращает: текст новости
  """
  res = requests.get(link)
  soup = BeautifulSoup(res.text)

  # Находим содержимое статьи
  post_content_body = soup.find('div', id='post-content-body')
  target_div = post_content_body.find('div', xmlns="http://www.w3.org/1999/xhtml")
  content = target_div.decode_contents()

  # Находим значение загловока
  h1_tag = soup.find('h1', {'data-test-id': 'articleTitle'})
  span_tag = h1_tag.find('span')
  title = span_tag.decode_contents()

  return { 'content': content, 'title': title }

def pretty_df(data):
  """Причесываем результаты для вывода"""
  # Создание DataFrame
  df = pd.DataFrame(data) 
  # Удаление столбца 'id'
  df = df.drop(columns=['id'])
  # Изменение порядка столбцов
  df = df.reindex(columns=['timePublished', 'title', 'link', 'content', 'score'])
  # Преобразование столбца в datetime; преобразование в московское время и извлечение только даты
  df['timePublished'] = pd.to_datetime(df['timePublished']).dt.tz_convert('Europe/Moscow').dt.date
  # Переименование столбцов
  df = df.rename(columns={
    'timePublished': 'дата',
    'title': 'заголовок',
    'link': 'ссылка на материал',
    'content': 'текст материала',
    'score': 'количество лайков'
    })
  
  return df

# Пример работы
# входящие данные
typed_query = ['python', 'анализ данных']
# преобразуем список в строку, т.к. habr.com вполне успешно работает с простым перечислением ключевых слов через запятую
query = ', '.join(typed_query)
# вторым параметром указываем количество страниц (по 5 результатов на страницу)
df = pretty_df(get_data(query, 2))
df

,дата,заголовок,ссылка на материал,текст материала,количество лайков
0,2020-04-17,В Яндекс.Облаке открыт доступ к анализу данных...,https://habr.com/news/497766,Сегодня на платформе Яндекс.Облако мы открывае...,23
1,2023-09-26,В России начался отбор на национальную школьну...,https://habr.com/news/763620,<p>Начался отбор на школьную олимпиаду DANO по...,5
2,2024-09-17,ИТМО провёл исследование open source в сферах ...,https://habr.com/news/844072,"<figure class=""full-width""><img data-blurred=""...",11
3,2024-12-10,Центральный университет и AIRI создают лаборат...,https://habr.com/news/865422,<p>Центральный университет вместе с некоммерче...,5
4,2024-12-27,"3 бесплатные нейросети, которые упрощают анали...",https://habr.com/news/870320,"<p>Не люблю громадные подборки сервисов, котор...",5
5,2021-01-27,Создание общей архитектуры для высокопроизводи...,https://habr.com/news/539454,Сегодня высокопроизводительные вычисления (<a ...,14
6,2021-08-22,ETL в анализе данных без перерывов на кофе и к...,https://habr.com/news/574110,"<p><img data-blurred=""true"" data-src=""https://...",8
7,2022-10-26,"Как понять, что пришло время внедрять платформ...",https://habr.com/news/695622,<p>Эффективные управленческие решения основаны...,2
